# RateBeer Data Set

This data set was obtained from RateBeer directly after requesting an API key. The data comes stored in a semi-structured and compressed/zipped .txt file. The relevant information will have to be parsed out into a more use-able format before cleaning and EDA.

There are almost 3 million (!) records. Each record spans 13 lines, 1 for each field and is separated by the next record by an empty line as shown in the sample below:

    beer/name: John Harvards Simcoe IPA
    beer/beerId: 63836
    beer/brewerId: 8481
    beer/ABV: 5.4
    beer/style: India Pale Ale &#40;IPA&#41;
    review/appearance: 4/5
    review/aroma: 6/10
    review/palate: 3/5
    review/taste: 6/10
    review/overall: 13/20
    review/time: 1157587200
    review/profileName: hopdog
    review/text: On tap at the Springfield, PA location. Poured a deep and cloudy orange (almost a copper) color with a small sized off white head. Aromas or oranges and all around citric. Tastes of oranges, light caramel and a very light grapefruit finish. I too would not believe the 80+ IBUs - I found this one to have a very light bitterness with a medium sweetness to it. Light lacing left on the glass.

After some exploration, it appears the records are all complete, with no missing information. Due to the size of the data set, there is a particular importance to optimizing storage/memory

## Parse Data

First, let's grab a single record using islice from the itertools module

In [2]:
from itertools import islice

with open('ratebeer.txt','r', encoding='latin-1') as file:
    # split into individual records using the empty line between each review record
    samples = list(islice(file, 14))
    for sample in samples:
        print(sample)


beer/name: John Harvards Simcoe IPA

beer/beerId: 63836

beer/brewerId: 8481

beer/ABV: 5.4

beer/style: India Pale Ale &#40;IPA&#41;

review/appearance: 4/5

review/aroma: 6/10

review/palate: 3/5

review/taste: 6/10

review/overall: 13/20

review/time: 1157587200

review/profileName: hopdog

review/text: On tap at the Springfield, PA location. Poured a deep and cloudy orange (almost a copper) color with a small sized off white head. Aromas or oranges and all around citric. Tastes of oranges, light caramel and a very light grapefruit finish. I too would not believe the 80+ IBUs - I found this one to have a very light bitterness with a medium sweetness to it. Light lacing left on the glass.





From here I've come up with an approach that should work. The code above showed a strategy for selecting each individual record as its own chunk by grabbing the 14 lines in a list with islice. Now we need to loop through all the chunks. For each record chunk, we can throw out the last line (it's empty) and then parse out the data values by splitting the key: value pairs on the ':' and keeping the value stored in a list of lists.

In [3]:
from itertools import islice

# empty list to house lists representing each record
records = []

with open('ratebeer.txt','r', encoding='latin-1') as file:
    while True:
        next_n_lines = list(islice(file, 14))
        
        if not next_n_lines:
            break
        
        # process block of 14 lines
        # remove last empty line from list
        next_n_lines.pop()
        
        # empty list to capture key value pairs after splitting
        record_list = []
        
        for field in next_n_lines:
            # each key and value is separated consistently by colon and space
            # we know that there may be colons in the review text, so only accept first occurence
            key_value = field.split(': ', 1)
            record_list.append(key_value[1].rstrip('\n'))
           
        records.append(record_list)

Now that we have the data in a list of lists, we can throw it into a dataframe. The benefit here of the structured data is that 

In [4]:
import pandas as pd

columns = ['beer_name',
           'beer_id',
           'beer_brewer_id',
           'beer_abv',
           'beer_style',
           'review_appearance',
           'review_aroma',
           'review_palate',
           'review_taste', 
           'review_overall',
           'review_time', 
           'review_user', 
           'review_text']

rb_df = pd.DataFrame(records, columns=columns)

In [5]:
rb_df.head()

,beer_name,beer_id,beer_brewer_id,beer_abv,beer_style,review_appearance,review_aroma,review_palate,review_taste,review_overall,review_time,review_user,review_text
0,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale &#40;IPA&#41;,4/5,6/10,3/5,6/10,13/20,1157587200,hopdog,"On tap at the Springfield, PA location. Poured..."
1,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale &#40;IPA&#41;,4/5,6/10,4/5,7/10,13/20,1157241600,TomDecapolis,On tap at the John Harvards in Springfield PA....
2,John Harvards Cristal Pilsner,71716,8481,5,Bohemian Pilsener,4/5,5/10,3/5,6/10,14/20,958694400,PhillyBeer2112,"UPDATED: FEB 19, 2003 Springfield, PA. I've ne..."
3,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Kölsch,2/5,4/10,2/5,4/10,8/20,1157587200,TomDecapolis,On tap the Springfield PA location billed as t...
4,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Kölsch,2/5,4/10,2/5,4/10,8/20,1157587200,hopdog,"On tap at the Springfield, PA location. Poured..."


In [7]:
rb_df.shape

(2924163, 13)

In [8]:
rb_df.to_pickle('ratebeer_raw_pickled')

## Clean Data

Now that the raw data has been captured in a more useful data structure, it's time to clean out the junk! A quick glance at the dataframe has the following ideas come to mind:

1. The beer_name field is bound to have to funky characters
2. The beer_abv column should be converted to float
3. There is evidence of some HTML characters in the beer_style column, we can replace those with their equivalents or remove them entirely
4. The string fractions for all the review ratings will have to get converted to float
5. Review time needs to be converted to date/time

As for the review text, I think lowercasing, removal of punctuation, and removal of non-alphabet characters will be sufficient. I want to keep that at a bare minimum for now, with the option to cut down more later when working on topic modeling tasks

In [3]:
import pandas as pd

In [4]:
rb_df = pd.read_pickle('ratebeer_raw_pickled')

Replacing funky HTML characters in beer names

In [5]:
import html

# Rogue HTML characters, convert to unicode equivalents
rb_df['beer_name'] = rb_df['beer_name'].apply(lambda x: html.unescape(x))

# dot character in place of single quotes i.e. Jake's, d'Oro, remove them
rb_df['beer_name'] = rb_df['beer_name'].str.replace('','')

Convert abv to numeric data type

In [6]:
rb_df['beer_abv'] = rb_df['beer_abv'].str.replace('-', '')
rb_df['beer_abv'] = pd.to_numeric(rb_df['beer_abv'], downcast='float')

Remove HTML characters from beer styles

In [7]:
rb_df['beer_style'] = rb_df['beer_style'].apply(lambda x: html.unescape(x))

Convert all the string fraction review fields to their decimal (float) equivalents. Going to use a function and loop to accomplish this one. 

In [8]:
def frac_to_float(frac_str):
    '''Takes a fraction string as input and returns its
    float value i.e. 3/5, 0.6'''
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [9]:
columns_with_fractions = ['review_appearance','review_aroma', 'review_palate', 'review_taste', 'review_overall']

for column in columns_with_fractions:
    rb_df[column] = rb_df[column].apply(lambda x: frac_to_float(x))

Convert the review time from unix to datetime 

In [10]:
rb_df['review_time'] = pd.to_datetime(rb_df['review_time'], unit='s')

A general cleaning of the reviews, removing capital letters, punctuation, unescape HTML

In [11]:
# import string
# import re

# rb_df['review_text'] = rb_df['review_text'].str.lower()
# rb_df['review_text'] = rb_df['review_text'].map(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

rb_df['review_text'] = rb_df['review_text'].apply(lambda x: html.unescape(x))

In [12]:
rb_df

,beer_name,beer_id,beer_brewer_id,beer_abv,beer_style,review_appearance,review_aroma,review_palate,review_taste,review_overall,review_time,review_user,review_text
0,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale (IPA),0.8,0.6,0.6,0.6,0.65,2006-09-07,hopdog,"On tap at the Springfield, PA location. Poured..."
1,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale (IPA),0.8,0.6,0.8,0.7,0.65,2006-09-03,TomDecapolis,On tap at the John Harvards in Springfield PA....
2,John Harvards Cristal Pilsner,71716,8481,5.0,Bohemian Pilsener,0.8,0.5,0.6,0.6,0.70,2000-05-19,PhillyBeer2112,"UPDATED: FEB 19, 2003 Springfield, PA. I've ne..."
3,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Kölsch,0.4,0.4,0.4,0.4,0.40,2006-09-07,TomDecapolis,On tap the Springfield PA location billed as t...
4,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Kölsch,0.4,0.4,0.4,0.4,0.40,2006-09-07,hopdog,"On tap at the Springfield, PA location. Poured..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924158,Oakhill Mendip Twister,12240,1028,6.3,Stout,0.6,0.7,0.6,0.7,0.75,2002-02-15,rauchbier,"Doesnt suggest it from the bottle label, but f..."
2924159,Oakhill Yeoman 1767 Strong Ale,5677,1028,5.0,Premium Bitter/ESB,0.8,0.5,0.6,0.6,0.60,2004-01-23,imdownthepub,"Cask conditioned at The Wild Goose, Combeignte..."
2924160,Oakhill Yeoman 1767 Strong Ale,5677,1028,5.0,Premium Bitter/ESB,0.8,0.4,0.6,0.6,0.65,2003-04-20,SilkTork,The original Oakhill Brewery was formed in 176...
2924161,Oakhill Yeoman 1767 Strong Ale,5677,1028,5.0,Premium Bitter/ESB,0.6,0.5,0.4,0.5,0.50,2002-03-18,omhper,Sampled cask conditioned. Amber coloured. Frui...


In [13]:
rb_df.to_pickle('ratebeer_pickled_clean')